In [5]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tqdm import tqdm

## Data preparation

In [6]:

data = pd.read_csv('induce-data-2019-08-08.csv').iloc[:, :]
vocab = ['C_E_F_T',
         'C_E_F_C',
         'C_E_F_O',
         'A_E_F_T',
         'A_E_F_O',
         'A_E_F_C',
         'G_E_F_C',
         'G_E_F_T',
         'G_E_F_O',
         'A_E_M_T',
         'A_E_M_O',
         'A_E_M_C',
         'G_E_M_O',
         'G_E_M_C',
         'G_E_M_T',
         'C_E_M_O',
         'C_E_M_C',
         'C_E_M_T',
         'C_H_F_CO',
         'C_H_F_CT',
         'C_H_F_OT',
         'G_H_F_OT',
         'G_H_F_CO',
         'G_H_F_CT',
         'A_H_F_CT',
         'A_H_F_OT',
         'A_H_F_CO',
         'C_H_M_CO',
         'C_H_M_CT',
         'C_H_M_OT',
         'A_H_M_CT',
         'A_H_M_OT',
         'A_H_M_CO',
         'G_H_M_OT',
         'G_H_M_CO',
         'G_H_M_CT', ]

labels = ['correct',
          'wrong',
          'type',
          'orientation',
          'color']

types = ['INTRO',
         'CORE',
         'FLEX',
         'TRIK',
         'DELY'
]

topics = ['cards',
          'animals',
          'geometry'
    
]

feat = ['type',
        'color',
        'orientation',
        'dual'
]

age = ['8-10','11-13']

# qts_id = ['A_E_F_C_INTRO', 'A_E_F_O_INTRO', 'A_E_F_T_INTRO', 'A_E_M_C_INTRO',
#         'A_E_M_O_INTRO', 'A_E_M_T_INTRO', 'A_H_F_CO_CORE', 'A_H_F_CO_DELY',
#         'A_H_F_CO_TRIK', 'A_H_F_CT_CORE', 'A_H_F_CT_DELY', 'A_H_F_CT_TRIK',
#         'A_H_F_OT_CORE', 'A_H_F_OT_DELY', 'A_H_F_OT_TRIK', 'A_H_M_CO_CORE',
#         'A_H_M_CO_FLEX', 'A_H_M_CT_CORE', 'A_H_M_CT_FLEX', 'A_H_M_OT_CORE',
#         'A_H_M_OT_FLEX', 'C_E_F_C_INTRO', 'C_E_F_O_INTRO', 'C_E_F_T_INTRO',
#         'C_E_M_C_INTRO', 'C_E_M_O_INTRO', 'C_E_M_T_INTRO', 'C_H_F_CO_CORE',
#         'C_H_F_CO_DELY', 'C_H_F_CO_TRIK', 'C_H_F_CT_CORE', 'C_H_F_CT_DELY',
#         'C_H_F_CT_TRIK', 'C_H_F_OT_CORE', 'C_H_F_OT_DELY', 'C_H_F_OT_TRIK',
#         'C_H_M_CO_CORE', 'C_H_M_CO_FLEX', 'C_H_M_CT_CORE', 'C_H_M_CT_FLEX',
#         'C_H_M_OT_CORE', 'C_H_M_OT_FLEX', 'G_E_F_C_INTRO', 'G_E_F_O_INTRO',
#         'G_E_F_T_INTRO', 'G_E_M_C_INTRO', 'G_E_M_O_INTRO', 'G_E_M_T_INTRO',
#         'G_H_F_CO_CORE', 'G_H_F_CO_DELY', 'G_H_F_CO_TRIK', 'G_H_F_CT_CORE',
#         'G_H_F_CT_DELY', 'G_H_F_CT_TRIK', 'G_H_F_OT_CORE', 'G_H_F_OT_DELY',
#         'G_H_F_OT_TRIK', 'G_H_M_CO_CORE', 'G_H_M_CO_FLEX', 'G_H_M_CT_CORE',
#         'G_H_M_CT_FLEX', 'G_H_M_OT_CORE', 'G_H_M_OT_FLEX']




def seq_to_int(qts,vocab, labels, types, feat, topics, n_steps, age):
    integ = list()
    for i,x in enumerate(qts):
        if i != n_steps:
            features = list()
            features.append(vocab.index(qts[i, 4]))
            features.append(labels.index(qts[i, 5]))
            features.append(qts[i, 2])
            features.append(types.index(qts[i, 9]))
            features.append(feat.index(qts[i, 10]))
            features.append(topics.index(qts[i, 6]))
            features.append(age.index(qts[i, 7]))
        else:
            features = list()
            features.append(vocab.index(qts[i, 4]))
            features.append(-1)
            features.append(qts[i, 2])
            features.append(types.index(qts[i, 9]))
            features.append(feat.index(qts[i, 10]))
            features.append(topics.index(qts[i, 6]))
            features.append(age.index(qts[i, 7]))
        integ.append(features)
    return integ

def split_sequence(data, n_steps, vocab, labels, types, feat, topics, age):
    X, Y = list(), list()
    users = list(dict.fromkeys(data.loc[:, "user"]))
    for u in users:
        sequence = data[data.user == u]
        for i in range(len(sequence)):
            end_idx = i + n_steps
            if end_idx > len(sequence)-1:
                break
            x = seq_to_int(sequence.iloc[i:end_idx+1, :].values, vocab, labels, types, feat, topics, n_steps, age)
            y = labels.index(str(sequence.iloc[end_idx, 5]))
            X.append(x)
            Y.append(y)
    return np.array(X), np.array(Y)


### Test split_sequence


In [7]:
print(split_sequence(data.iloc[:20, :], 10, vocab, labels, types, feat, topics, age))


(array([[[ 0,  0,  0,  0,  0,  0,  1],
        [ 1,  0,  1,  0,  1,  0,  1],
        [ 2,  0,  2,  0,  2,  0,  1],
        [ 3,  0,  3,  0,  0,  1,  1],
        [ 4,  0,  4,  0,  2,  1,  1],
        [ 5,  1,  5,  0,  1,  1,  1],
        [ 6,  0,  6,  0,  1,  2,  1],
        [ 7,  0,  7,  0,  0,  2,  1],
        [ 8,  0,  8,  0,  2,  2,  1],
        [ 9,  0,  9,  0,  0,  1,  1],
        [10, -1, 10,  0,  2,  1,  1]],

       [[ 1,  0,  1,  0,  1,  0,  1],
        [ 2,  0,  2,  0,  2,  0,  1],
        [ 3,  0,  3,  0,  0,  1,  1],
        [ 4,  0,  4,  0,  2,  1,  1],
        [ 5,  1,  5,  0,  1,  1,  1],
        [ 6,  0,  6,  0,  1,  2,  1],
        [ 7,  0,  7,  0,  0,  2,  1],
        [ 8,  0,  8,  0,  2,  2,  1],
        [ 9,  0,  9,  0,  0,  1,  1],
        [10,  0, 10,  0,  2,  1,  1],
        [11, -1, 11,  0,  1,  1,  1]],

       [[ 2,  0,  2,  0,  2,  0,  1],
        [ 3,  0,  3,  0,  0,  1,  1],
        [ 4,  0,  4,  0,  2,  1,  1],
        [ 5,  1,  5,  0,  1,  1,  1],
       

## The model


In [17]:

# n_steps = 10

X = list()
y = list()

for i in tqdm(range(5, 10)):
    X_seq, y_seq = split_sequence(data, i, vocab, labels, types, feat, topics, age)
    for x in X_seq:
        X.append(x)
    for _y in y_seq:
        y.append(_y)
y = np_utils.to_categorical(y)
# X = np_utils.to_categorical(X)


100%|██████████| 5/5 [00:33<00:00,  6.78s/it]


In [18]:
n_features = 7
# X = X.reshape((X.shape[0], X.shape[1], n_features*X.shape[-1]))
# print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
print('train set size: '+str(len(y_train)))
print('test set size: '+str(len(y_test)))

model = Sequential()
model.add(LSTM(50, input_shape=(None, n_features)))
# model.add(Dropout(0.2))
# model.add(LSTM(X.shape[1], return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(X.shape[1]))
# model.add(Dropout(0.2))

model.add(Dense(5))
model.compile(optimizer='adam', loss='mse', metrics=['categorical_accuracy'])

train set size: 55858
test set size: 6207


### Training

In [20]:
def train_generator():
    while True:
        idx = np.random.choice(np.arange(len(y_train)), 1, replace=False)[0]
        x_train = X_train[idx]
        _y_train = y_train[idx]
        yield np.array([x_train]), np.array([_y_train])

model.fit_generator(train_generator(), steps_per_epoch=10000, epochs=2000, verbose=1)

Epoch 1/2000
1000/1000 [==============================] - 22s 22ms/step - loss: 0.1197 - categorical_accuracy: 0.4590
Epoch 2/2000
1000/1000 [==============================] - 21s 21ms/step - loss: 0.1199 - categorical_accuracy: 0.4460
Epoch 3/2000
 528/1000 [==============>...............] - ETA: 10s - loss: 0.1204 - categorical_accuracy: 0.4527

KeyboardInterrupt: 

### Validation

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

print(X_test[0])
print(model.predict(np.array([X_test[0]])))
print(np.array([X_test[1]]))
print(y_test[0])

